In [1]:
from IPython.display import display, Math
from sympy import *
import sympy as sp
import numpy as np
import scipy as si

1. Gauss Elimination
2. Gauss Elimination with Backsubstitution
3. Gauss Elimination with Backsubstitution and Pivot
4. Gauss-Jordan Elimination
5. Gauss Seidel iterative method
6. Jacobi iterative method
7. Cholesky Decomposition
8. LU Decomposition
9. QR Decomposition 
10. Power Iteration and Inverse Power Iteration

 ## Gauss elimination method

In [2]:
def gauss(a):
    m, n = a.shape               
    M = a.copy()
     
    for k in range(n):                        
        for j in range(k+1,n):                      
            q = float(M[j][k]) / M[k][k]  # lambda
            for m in range(k, n): 
                M[j][m] -=  q * M[k][m]        
    return M

In [3]:
A = np. array([[  1.,  2., -1.,  1.],
               [ -1.,  1.,  2., -1.],
               [  2., -1.,  2.,  2.],
               [  1.,  1., -1.,  2.]])

b = gauss(A)
b,np.linalg.det(b),np.linalg.det(A)

(array([[ 1.        ,  2.        , -1.        ,  1.        ],
        [ 0.        ,  3.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  5.66666667,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 17.0,
 17.0)

* determinant
* 

### Gauss Elimination with Backsubstitution

In [4]:
def gauss_elmbk(a, b):
    m, n = a.shape               
    b = b.reshape(n,1)
    M = np.hstack((a,b))                      #augmented_mat
    const = M.copy()
    #display(Math(r'A|B_b = '+latex(sp.Matrix(M))))
    for k in range(n):                       
        for j in range(k+1,n):                      
            q = float(M[j][k]) / M[k][k]       # lambda
            for m in range(k, n+1):    # Forward Elimination - Subtracting rows
                M[j][m] -=  q * M[k][m] 
    display(Math(r'A|B_{Gauss-back} = '+latex(sp.Matrix(const.round(2)))+
                 r'\rightarrow'+latex(sp.Matrix(np.round(M,2)))))
    
    x = np.zeros(n)
    x[n-1] = float(M[n-1][n]) / M[n-1][n-1]  
    for i in range (n-1,-1,-1):                 # Backwards Substitution 
        z = 0.0                                     
        for j in range(i+1,n):                     
            z = z  + float(M[i][j])*x[j]   
        x[i] = float(M[i][n] - z) / M[i][i]
    return x

### Using Gauss elimination method find the inverse of matrix

In [28]:
def gauss_elmbk_inv(a):
    m, n = a.shape               
    b = np.identity(len(a))
    M = np.hstack((a,b))                        #augmented_mat
    const1 = M.copy()
    #display(Math(r'A|I_b = '+latex(sp.Matrix(M))))

    for k in range(n+len(a)):                  
        for j in range(k+1,n):                      
            q = float(M[j][k]) / M[k][k]        # lambda
            for m in range(k, n+len(a)):        # Forward Elimination - Subtracting rows 
                M[j][m] -=  q * M[k][m]  

    #display(Math(r'A|I_b = '+latex(sp.Matrix(np.round(M,2)))))
    const2 = M.copy()

    for i in range(0, n):
        M[i] = M[i] / M[i][i]        
        for j in [k for k in range(0, n) if k != i]:
            M[j] = M[j] - M[i] * M[j][i]
    display(Math(r'A|I_{Gauss-back} = '+latex(sp.Matrix(const1))+
                 r'\rightarrow'+latex(sp.Matrix(np.round(const2,2)))))
    display(Math(r'I|A^{-1}_{Gauss-back} = '+latex(sp.Matrix(np.round(M,2)))))
    return M[:, n:]

### Gauss Elimination with Backsubstitution and Pivot

In [29]:
def gauss_pivot(a, b):
    m, n = a.shape               
    b = b.reshape(n,1)
    M = np.hstack((a,b))                   #augmented_mat
    #display(Math(r'A|B_p = '+latex(sp.Matrix(M))))
    const = M.copy()
    def swaprows(v,i,j):
        if len(v.shape) == 1: v[i],v[j] = v[j],v[i]
        else:   v[[i,j],:] = v[[j,i],:]
    s = np.zeros(n)
    for i in range(n):  s[i] = max(np.abs(M[i,:]))
    for k in range(n):       
        p = np.argmax(np.abs(a[k:n,k])/s[k:n]) + k   # row interchange, if needed
        if p != k: swaprows(M,k,p)
        for j in range(k+1,n):  
            q = float(M[j][k]) / M[k][k]        # lambda
            for m in range(k, n+1): M[j][m] -=  q * M[k][m] # Forward Elimination 
    display(Math(r'A|B_{Gauss-Pivot} = '+latex(sp.Matrix(np.round(const,2)))+
                 r'\rightarrow'+latex(sp.Matrix(np.round(M,2)))))
    x = np.zeros(n)
    x[n-1] =float(M[n-1][n]) / M[n-1][n-1]  
    for i in range (n-1,-1,-1):                         # Backwards Substitution         
        z = 0.0                                     
        for j in range(i+1,n): z = z  + float(M[i][j])*x[j]   
        x[i] = float(M[i][n] - z) / M[i][i]    
    return x 

## Gauss-Jordan Elimination

In [30]:
def gauss_jordan(a, b):
    m, n = a.shape               
    b = b.reshape(n,1)
    M = np.hstack((a,b))                      #augmented_mat
    #display(Math(r'A|B_j = '+latex(sp.Matrix(M))))
    const = M.copy()
    ol = [[0, m - 1, 1], [m - 1, 0, -1]]               #outer_loop
    for d in range(2):
        for i in range(ol[d][0], ol[d][1], ol[d][2]):
            il = [[i + 1, m, 1], [i - 1, -1, -1]]                #inner_loop
            for j in range(il[d][0], il[d][1], il[d][2]):
                k = (-1) * M[j, i] / M[i, i]
                temp_row = M[i, :] * k
                M[j, :] += temp_row
    for i in range(0, m):
        M[i, :] = M[i, :] / M[i, i]
    display(Math(r'A|B_{Gauss-Jordan} = '+latex(sp.Matrix(np.round(const,2)))+
                 r'\rightarrow'+latex(sp.Matrix(np.round(M,2)))))
    return M[:, n]

### Using Gauss Jordan elimination find the inverse of matrix

In [31]:
def gauss_jordan_inv(a):
    m, n = a.shape              
    b = np.identity(len(a))
    M = np.hstack((a,b))           #augmented_matrix
    #display(Math(r'A|I_j = '+latex(sp.Matrix(M))))
    const1 = M.copy()
    ol = [[0, m - 1, 1], [m - 1, 0, -1]]           #outer_loop
    for d in range(2):
        for i in range(ol[d][0], ol[d][1], ol[d][2]):
            il = [[i + 1, m, 1], [i - 1, -1, -1]]       #inner_loop
            for j in range(il[d][0], il[d][1], il[d][2]):
                k = (-1) * M[j, i] / M[i, i]
                temp_row = M[i, :] * k
                M[j, :] += temp_row
    const2 = M.copy()
    for i in range(0, m):
        M[i, :] = M[i, :] / M[i, i]
    display(Math(r'A|I_{Gauss-Jordan} = '+latex(sp.Matrix(const1))+
                 r'\rightarrow'+latex(sp.Matrix(const2.round(2)))))
    display(Math(r'I|A^{-1}_{Gauss-Jordan} = '+latex(sp.Matrix(np.round(M,2)))))
    return M[:, n:]

In [32]:
A = np. array([[  1.,  2., -1.,  1.],
               [ -1.,  1.,  2., -1.],
               [  2., -1.,  2.,  2.],
               [  1.,  1., -1.,  2.]])

b = np.array([ 6., 3., 14., 8.])

x_gb = gauss_elmbk(A, b)
x_gp = gauss_pivot(A, b)
x_gj = gauss_jordan(A, b)
x = np.linalg.solve(A,b)

display(Math(r'X_{Gauss-back} = '+latex(sp.Matrix(np.round(x_gb,2)))+
             r'X_{Gauss-Pivot} = '+latex(sp.Matrix(np.round(x_gp,2)))+
             r'X_{Gauss-Jordan} = '+latex(sp.Matrix(np.round(x_gj,2)))+
             r'X = '+latex(sp.Matrix(np.round(x,2)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [33]:
inv_gb = gauss_elmbk_inv(A)
inv_gj = gauss_jordan_inv(A)
inv = si.linalg.inv(A)

display(Math(r'A^{-1}_{Gauss-back} = '+latex(sp.Matrix(np.round(inv_gb,2)))+
             r'A^{-1}_{GaussJordan} = '+latex(sp.Matrix(np.round(inv_gj,2)))+
             r'A^{-1} = '+latex(sp.Matrix(np.round(inv,2)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Gauss Seidel iterative method 

In [11]:
def gauss_seidel(a,b,limit=1000):
    m,n = a.shape
    x = np.zeros_like(b)
    for it_count in range(1, limit):
        x_new = np.zeros_like(x)
        for i in range(a.shape[0]):
            s1 = np.dot(a[i, :i], x_new[:i])
            s2 = np.dot(a[i, i + 1 :], x[i + 1 :])
            x_new[i] = (b[i] - s1 - s2) / a[i, i]
        if np.allclose(x, x_new, rtol=1e-8):    # check for convergence
            break
        x = x_new
    return x

## Jacobi Iterative method

In [12]:
def jacobi_iterative(a,b,limit = 100):
    m,n = a.shape
    x = np.zeros_like(b)
    for it_count in range(limit+1):   
        x_new = np.zeros_like(x)
        for i in range(a.shape[0]):
            s1 = np.dot(a[i, :i], x[:i])
            s2 = np.dot(a[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / a[i, i]
            #if x_new[i] == x_new[i-1]:   # check for convergence
            #    break
        if np.allclose(x, x_new, atol=1e-10, rtol=0.):
            break
        x = x_new
    return x

In [13]:
A = np.array([[10., -1.,   2.,  0.],
              [-1., 11.,  -1.,  3.],
              [ 2., -1.,  10., -1.],
              [ 0.,  3.,  -1.,  8.]])

b = np.array([6., 25., -11., 15.])

X_j = jacobi_iterative(A,b)
X_gs = gauss_seidel(A,b)
X = si.linalg.solve(A,b)

inv_j = jacobi_iterative(A,b=np.identity(len(A)))
inv_gs = gauss_seidel(A,b=np.identity(len(A)))
inv = si.linalg.inv(A)

display(Math(r'X_{Gauss-Siedel} = '+latex(sp.Matrix(X_gs.round(2)))+
             r'X_{Jacobi} = '+latex(sp.Matrix(X_j.round(2)))+
             r'x = '+latex(sp.Matrix(np.round(X,2)))))

display(Math(r'A^{-1}_{Gauss-Siedel} = '+latex(sp.Matrix(inv_gs.round(2)))+
             r'A^{-1}_{Jacobi} = '+latex(sp.Matrix(inv_j.round(2)))+
             r'A^{-1}= '+latex(sp.Matrix(np.round(inv, 2)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Cholesky Decomposition

In [14]:
def cholesky_decomposition(A):
    A = np.double(A)
    n = A.shape[0]
    L = np.zeros_like(A, dtype=float)

    for i in range(n):
        for j in range(i + 1):
            if i == j:
                temp = A[i, i] - np.sum(L[i, :i] ** 2)
                if temp <= 0:
                    raise ValueError("Matrix is not positive definite.")
                L[i, i] = np.sqrt(temp)
            else:
                L[i, j] = (A[i, j] - np.sum(L[i, :j] * L[j, :j])) / L[j, j]

    return L.T

### Using Cholesky decomposition method solve Ax = b

In [15]:
def cholesky_x(A, b):
    L = cholesky_decomposition(A)
    y = np.linalg.solve(L.T, b)
    x = np.linalg.solve(L, y)
    return x

In [16]:
l = cholesky_decomposition(A)
L = si.linalg.cholesky(A)

x_c = cholesky_x(A, b)
x = si.linalg.solve(A,b)

inv_c = cholesky_x(A, np.identity(len(A)))
inv = si.linalg.inv(A)

display(Math(r'L_c = '+latex(sp.Matrix(l.round(2)))+
             r'L = '+latex(sp.Matrix(L.round(2)))))

display(Math(r'x_c = '+latex(sp.Matrix(np.round(x_c,2)))+
             r'x = '+latex(sp.Matrix(np.round(x,2)))))

display(Math(r'A^{-1}_c = '+latex(sp.Matrix(np.round(inv_c,2)))+
             r'A^{-1} = '+latex(sp.Matrix(np.round(inv,2)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## LU decomposition

In [17]:
def lu_decomposition(a):
    a = np.double(a)
    n = a.shape[0]
    L = np.eye(n, dtype=np.double)
    U = np.double(np.copy(a))
    P = np.eye(n, dtype=np.double)  
    for k in range(n - 1):
        max_row = np.argmax(np.abs(U[k:, k])) + k
        if max_row != k:
            U[[k, max_row]] = U[[max_row, k]]
            P[[k, max_row]] = P[[max_row, k]]
            L[[k, max_row], :k] = L[[max_row, k], :k]
        for i in range(k + 1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] = U[i, k:] - L[i, k] * U[k, k:]
    return P.T, L, U

### Using LU decomposition method solve Ax = b 

In [18]:
def PLU_x(a, b):
    P,L,U = lu_decomposition(a)
    y = np.linalg.solve(L, np.dot(si.linalg.inv(P),b))
    x = np.linalg.solve(U, y)
    return x

In [19]:
p,l,u = lu_decomposition(A)
P,L,U = si.linalg.lu(A)

X_lu = PLU_x(A,b)
X = si.linalg.solve(A,b)

inv_lu = PLU_x(A,np.identity(len(A)))
inv = si.linalg.inv(A)

display(Math(r'P_{LU} = '+latex(sp.Matrix(p.round(2)))+
             r'P = '+latex(sp.Matrix(P.round(2)))))

display(Math(r'L_{LU} = '+latex(sp.Matrix(l.round(2)))+
             r'L = '+latex(sp.Matrix(L.round(2)))))

display(Math(r'U_{LU} = '+latex(sp.Matrix(u.round(2)))+
             r'U = '+latex(sp.Matrix(U.round(2)))))

display(Math(r'X_{LU} = '+latex(sp.Matrix(X_lu.round(2)))+
             r'x = '+latex(sp.Matrix(np.round(X,2)))))

display(Math(r'A^{-1}_{LU} = '+latex(sp.Matrix(inv_lu.round(2)))+
             r'A^{-1} = '+latex(sp.Matrix(np.round(inv,2)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## QR decomposition

In [20]:
def QR_decomposition(A):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        v = A[:, j]
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], A[:, j])
            v = v - R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]

    return -Q, -R

### Using QR decomposition method solve Ax = b

In [21]:
def QR_x(A, b):
    Q, R = QR_decomposition(A)
    Qt_b = np.dot(Q.T, b)
    n = R.shape[1]
    x = np.zeros(n)

    for i in range(n - 1, -1, -1):
        x[i] = (Qt_b[i] - np.dot(R[i, i + 1:], x[i + 1:])) / R[i, i]

    return x

### Using QR decomposition method from Ax = b solve $A^{-1}$

In [22]:
def QR_inv(A):
    n = A.shape[0]
    b = np.identity(n)
    A_inv = np.zeros((n, n))

    for i in range(n):
        column_i = b[:, i]
        A_inv[:, i] = QR_x(A, column_i)

    return A_inv

In [23]:
q,r = QR_decomposition(A)
Q, R = np.linalg.qr(A)

x_QR = QR_x(A, b)
x = si.linalg.solve(A,b)

inv_QR = QR_inv(A)
inv = si.linalg.inv(A)

display(Math(r'Q_{QR} = '+latex(sp.Matrix(q.round(2)))+
             r'Q = '+latex(sp.Matrix(Q.round(2)))))

display(Math(r'R_{QR} = '+latex(sp.Matrix(r.round(2)))+
             r'R = '+latex(sp.Matrix(R.round(2)))))

display(Math(r'x_{QR} = '+latex(sp.Matrix(np.round(x_QR,2)))+
             r'x = '+latex(sp.Matrix(np.round(x,2)))))

display(Math(r'A^{-1}_{QR} = '+latex(sp.Matrix(np.round(inv_QR,2)))+
             r'A^{-1} = '+latex(sp.Matrix(np.round(inv,2)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Power Iteration and Inverse Iteration:

In [24]:
def power(M, tol=1e-12): 
    X = np.ones(len(M))
    den = (X.T).dot(X)
    Y = M.dot(X)
    c,lambda_0,d = 0,0,1

    while abs(d) > tol:
        c += 1
        num = (X.T).dot(Y)
        lambda_1 = num / den
        d = lambda_1 - lambda_0
        lambda_0 = lambda_1
        den = num
        Y = M.dot(Y)
    E = Y / np.linalg.norm(Y)
    return lambda_1,E

def inverse_power(M,tol=1e-12):
    M_inv = np.linalg.inv(M)
    X = np.ones(len(M_inv))
    den = (X.T).dot(X)
    Y = M_inv.dot(X)
    c,lambda_0,d = 0,0,1

    while abs(d) > tol:
        c += 1
        num = (X.T).dot(Y)
        lambda_1 = num / den
        d = lambda_1 - lambda_0
        lambda_0 = lambda_1
        den = num
        Y = M_inv.dot(Y)
    E = Y / np.linalg.norm(Y)
    return lambda_1, E

In [25]:
M = np.array([[ 4.,-1., 0., 1.],
              [-1., 6.,-2., 0.],
              [ 0.,-2., 3., 2.],
              [ 1., 0., 2., 4.]])

L, E = power(M)
Li, Ei = inverse_power(M)
l, v = np.linalg.eig(M)
v = v.T

display(Math(r'\lambda_{max} : '+latex(L.round(2))+
             r'\quad X : '+latex(sp.Matrix(E.round(2)))+
             r'\lambda_{min} : '+latex(np.round(1/Li,2))+
             r'\quad X : '+latex(sp.Matrix(Ei.round(2)))))

for i in range(len(l)):
    display(Math(f'\lambda_{i+1} : '+latex(l[i].round(2))+
             r'\quad X : '+latex(sp.Matrix(v[i].round(2)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Jacobi Method to find all eigen values & eigen vectors

In [26]:
def jacobi(A, tol=1e-12, max_iter=1e12):
    a = np.copy(A)

    def maxElem(a):  # Find largest off-diagonal element a[k,l]
        n = len(a)
        aMax = 0.0
        for i in range(n - 1):
            for j in range(i + 1, n):
                if abs(a[i, j]) >= aMax:
                    aMax = abs(a[i, j])
                    k = i
                    l = j
        return aMax, k, l

    def rotate(a, p, k, l):  # Rotate to make a[k,l] = 0
        n = len(a)
        aDiff = a[l, l] - a[k, k]
        if abs(a[k, l]) < abs(aDiff) * 1.0e-36:
            t = a[k, l] / aDiff
        else:
            phi = aDiff / (2.0 * a[k, l])
            t = 1.0 / (abs(phi) + (phi**2 + 1.0)**0.5)
            if phi < 0.0:
                t = -t
        c = 1.0 / (t**2 + 1.0)**0.5
        s = t * c
        tau = s / (1.0 + c)
        temp = a[k, l]
        a[k, l] = 0.0
        a[k, k] = a[k, k] - t * temp
        a[l, l] = a[l, l] + t * temp
        for i in range(k):  # Case of i < k
            temp = a[i, k]
            a[i, k] = temp - s * (a[i, l] + tau * temp)
            a[i, l] = a[i, l] + s * (temp - tau * a[i, l])
        for i in range(k + 1, l):  # Case of k < i < l
            temp = a[k, i]
            a[k, i] = temp - s * (a[i, l] + tau * a[k, i])
            a[i, l] = a[i, l] + s * (temp - tau * a[i, l])
        for i in range(l + 1, n):  # Case of i > l
            temp = a[k, i]
            a[k, i] = temp - s * (a[l, i] + tau * temp)
            a[l, i] = a[l, i] + s * (temp - tau * a[l, i])
        for i in range(n):  # Update transformation matrix
            temp = p[i, k]
            p[i, k] = temp - s * (p[i, l] + tau * p[i, k])
            p[i, l] = p[i, l] + s * (temp - tau * p[i, l])

    n = len(a)
    p = np.identity(n)  # Initialize transformation matrix
    for i in range(int(max_iter)):  # Jacobi rotation loop
        aMax, k, l = maxElem(a)
        if aMax < tol:
            eigenvalues = np.diagonal(a)
            return eigenvalues, p
        rotate(a, p, k, l)
    
    print('Jacobi method did not converge after', max_iter, 'iterations')
    return None

In [27]:
L, V = jacobi(M)
L = np.sort(L)
V = np.sort(V)

l, v = np.linalg.eig(M)
l = np.sort(l)
v = np.sort(v)

for i in range(len(l)):
    display(Math(f'\lambda_{i+1} : '+latex(np.round(L[i],2))+
                 f'\quad X_{i+1} : '+latex(sp.Matrix(V[i].round(2)))+
                 f'\lambda_{i+1} : '+latex(l[i].round(2))+
                 f'\quad X_{i+1} : '+latex(sp.Matrix(v[i].round(2)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>